#Volatility strategies#

Volatility trading has been gaining quite some attention lately and for a good reason. There is definately some money to be made there, but you need to be careful. Trading volatility often involves shorting it, which is a pretty risky business as 20% daily volatility spike is not that uncommon.

Shorting the short-term volatility etn VXX may seem like a great idea when you look at the chart from quite a distance. Due to the contango in the volatility futures, the etn experiences quite some headwind most of the time and looses a little bit its value every day. This happens due to daily rebalancing, for more information please look into the prospect.

#Strategy thesis#
VXX experiences most drag when the futures curve is in a steep contango. The futures curve is approximated by the VIX-VXV relationship. We will short VXX when VXV has an unusually high premium over VIX.

#Get the data#
From Yahoo Finance we need the volatility ETNs VXX and VXZ and SPY. From CBOE we will get the volatility indices VIX (spot) and VXV (1-month)
As usual, the twp toolbox offers help here. 

In [1]:
%pylab inline
figsize(15,8)

Populating the interactive namespace from numpy and matplotlib


In [2]:
#import libraries
import tradingWithPython as twp # general functions
import tradingWithPython.lib.cboe as cboe # cboe data
import tradingWithPython.lib.yahooFinance as yf  # yahooFinance data
import pandas as pd

In [3]:
# get data
spy = yf.getHistoricData('SPY')['adj_close']
vxx = yf.getHistoricData('VXX')['adj_close']
vxz = yf.getHistoricData('VXZ')['adj_close']
vix = cboe.getHistoricData('VIX') 
vxv = cboe.getHistoricData('VXV')


Got 6049 days of data
Got 2018 days of data
Got 2004 days of data


In [7]:
type(vxv)

pandas.core.frame.DataFrame

In [8]:
type(spy)

pandas.core.series.Series

In [4]:
#combine all data to a single DataFrame
df = pd.DataFrame({'SPY':spy,'VXX':vxx,'VXZ':vxz,'VIX':vix,'VXV':vxv}).dropna() #直接合并，不同的index范围，直接被dropna了。
df.plot()

ValueError: could not broadcast input array from shape (2309) into shape (3296)

#Research relationship VIX-VXV#
The ratio of VIX to VXV gives a good indication of the steepness of the fututures curve. The two are closely (co)related. Working with VIX/VXV ratio has been discussed in the blogsphere for some time, the best place to start is probably the [VIX an More blog](http://vixandmore.blogspot.nl/search/label/VIX%3AVXV).

In [ ]:
vol = pd.DataFrame({'VIX':vix,'VXV':vxv}).dropna() # volatility DataFrame
vol.plot()

I'll assume that the relationship between VXV and VIX is a second order polynome, our model will be VXV = a*VIX^2+b*VIX+c . I aslo could have used a simple linear first-order fit. A higher order (3 or more) fit is not favorable here as it is just to sensitive to overfitting.

In [ ]:
x = vol.VIX.values # independant variable vector
y = vol.VXV.values # dependant variable vector

order = 2 # fit order. 1-linear , 2-quadratic

p = np.polyfit(x,y,order)
print 'Poly coefficients:' ,  p

xx = np.linspace(10,80) # linear vector for evaluating fit


plot(x,y,'go')

w = 250
plot(x[-w:],y[-w:],'ro') # plot last w days in red
plot(xx,np.polyval(p,xx),linewidth=3) # plot computed fit

grid(True)

legend(('all days','last %i days' % w))
title('VIX-VXV relationship')

xlabel('VIX')
ylabel('VXV')


now we can create a function that estimates VXV based on VIX value. The difference between the current VXV and its estimated value will be our indicator *delta*

In [ ]:
def estimateVXV(vix, p):
    ''' estimate VXV based on VIX value '''
    vxv = pd.Series(index=vix.index,data=np.polyval(p, vix))
    return vxv


delta = (vxv-estimateVXV(vix,p)).dropna() # create delta indicator
delta.plot()
title('Delta')

Let's simulate a couple of strategies. Individual results are saved to (temporary) *pnl* Series. The results of all strategies will be saved to the *PNL* DataFrame (capital letters)

In [ ]:
PNL = pd.DataFrame(index=delta.index) # init PNL with same dates as delta

In [ ]:
#-------dumbVXX strategy---------------
# just sell VXX and rebalance each day to keep capital constant
ret = vxx.pct_change()
pnl = -ret # maintain short position
PNL['dumbVXX'] = pnl


#--------SPY buy&hold----------------
ret = spy.pct_change()
pnl = ret*3 # multiplying by 3 here because VXX has beta of about 3. .
PNL['SPY'] = pnl


#--------smartVXX strategy------------
# hold short VXX position when delta is positive (VXV is higher than it should be compared to VIX, the futures are in stronger than 
# normal contango.

threshDelta = 0 # delta threshold, strategy parameter
tradeDir = 1*(delta > threshDelta) # trading direction. Multiply by 1 to convert [True/False] to numeric values [1/0]
tradeDir = -tradeDir.fillna(0) # fill nans with zeros, '-' is for the short position

ret = vxx.pct_change() # daily returns
pnl = ret * tradeDir.shift(1) # don't forget to shift(1), todays position has effect on tomorrows return.

PNL['smartVXX'] = pnl


#-------smartSPY_VXX pair-----------
# a near market-neutral pair would be a combination of 3xSPY+1xVXX. Let's trade it with same signals
threshDelta = 0 # delta threshold, strategy parameter
ret = vxx.pct_change()+3*spy.pct_change()
ret = ret*2 # increase leverage to achieve same beta as VXX
tradeDir = 1*((delta > threshDelta) & (vix.reindex(delta.index) < 25))# trading direction. Multiply by 1 to convert [True/False] to numeric values [1/0]
tradeDir = -tradeDir.fillna(0) # fill nans with zeros, '-' is for the short position
pnl = ret * tradeDir.shift(1) # don't forget to shift(1), todays position has effect on tomorrows return.
PNL['smartSPY_VXX'] = pnl

#------------------------------------
PNL = PNL.dropna() # remove rows with mising entries. 

PNL.cumsum().plot() 

#---------print statistics
print '----Sharpe ratio----'
print twp.sharpe(PNL)
print '----Profit ratio----'
print twp.profitRatio(PNL)


It is clear that the strategies using delta are performing much better than VXX and SPY buy-and-hold in terms of return/drawdown. A lower drawdown will enable to increase leverage or simply reduce the stress of watching a portfolio go down.

#More precise simulation#

Above simulations are just a quick exploration of the possible strategies. I will now calculate the `smartVXX` strategy in more detail. We'll be using a walk-forward again.

In [ ]:
# create estimator class for  VXV as a function of VIX. Same functionality as before,
# but now nicely wrapped inside a class

class VXV_estimator(object):
    ''' class for estimating VXV based on VIX value '''
    def __init__(self,order=2):
        ''' order: polynome order of the model '''
        self.order = order
        
        self.p = None
        
    def fit(self,vix,vxv):
        ''' compute model based on vix and vxv values '''
        vol = pd.DataFrame({'vix':vix,'vxv':vxv}).dropna() # create dataframe, drop missing rows
               
        x = vol.vix.values # independant variable vector
        y = vol.vxv.values # dependant variable vector
        
        self.p = np.polyfit(x,y,self.order)
       
        
    def estimate(self,vix):
        ''' estimate vxv based on model '''
        vxv = pd.Series(index=vix.index,data=np.polyval(self.p, vix))
        return vxv
    
#----test estimator class
    
e = VXV_estimator(order=2)
e.fit(vol.VIX,vol.VXV)

delta = (vxv-e.estimate(vix)).dropna() # create delta indicator
delta.plot()

In [ ]:
# pos2pnl is same as in twp toolbox, only simplified to work with Series.
def pos2pnl(price,position , ibTransactionCost=False ):
    """
    calculate pnl based on price and position
    Inputs:
    ---------
    price: series  of price
    position: number of shares at each time. Column names must be same as in price
    ibTransactionCost: use bundled Interactive Brokers transaction cost of 0.005$/share

    Returns a portfolio DataFrame. 
    Output is *cumulative* pnl
    """

    delta=position.diff()
    port = pd.DataFrame(index=price.index)

    
    port['cash'] = (-delta*price).cumsum()
    port['stock'] = (position*price)

    if ibTransactionCost:
        tc = -0.005*position.diff().abs() # basic transaction cost
        tc[(tc>-1) & (tc<0)] = -1  # everything under 1$ will be ceil'd to 1$
        port['tc'] = tc.cumsum()
    else:
        port['tc'] = 0.

    port['total'] = port['stock']+port['cash']+port['tc']

    return port



def simulate(vxx,delta, threshDelta = 0, bet=10000):
    ''' simulate smartVXX strategy '''
    
    tradeDir = 1*(delta > threshDelta) # trading direction. 
    
    
    tradeDir = -tradeDir.fillna(0) # fill nans with zeros, '-' is for the short position
    pos = tradeDir*bet/vxx # number of shares held
    
    
    pnl = pos2pnl(vxx,pos, ibTransactionCost=True)
    return pnl

#---test function

pnl = simulate(df.VXX, delta)

pnl.plot()


In [ ]:
def generateFWD(data,nTrain = 250, nTest=50):
    ''' 
    Walk forward generator. 
    '''
    nTotal = nTrain+nTest
    
    for tp in xrange(nTrain,len(data) - nTest, nTest) :  # xrange() is a little more efficient than range()
        # tp is a 'testpoint' which is at in-sample/out-of-sample boundary
        yield (data.iloc[tp-nTrain:tp,:], data.iloc[tp:tp+nTest]) # return test and train datasets

        
# create datasets. Each element contains a [train,test] pair
# train is in-sample data, test is out-of-sample data
datasets = [[train,test] for train,test in generateFWD(df)]

print 'Created %i datasets' % len(datasets)

In [ ]:
# perform walk-forward test
pnl = []

est = VXV_estimator(order=2)

for dataset in datasets:
    train = dataset[0]
    test = dataset[1]
    
    est.fit(train.VIX,train.VXV)
    delta = (test.VXV-est.estimate(test.VIX)).dropna() # create delta indicator
    
    pnl.append(simulate(test.VXX,delta, threshDelta = 0)['total'].diff()) 
    # .diff() is needed because pos2pnl returns cumulative result.
    
pnl = pd.concat(pnl).fillna(0) # convert list of dataframes to one dataframe
pnl.cumsum().plot()
title('smartVXX strategy with transaction cost')

print 'Sharpe:', twp.sharpe(pnl)
print 'Profit ratio:', twp.profitRatio(pnl)

With a more precise backtest the strategy loses some sharpe and there are more inactive periods. The profit ratio is still not bad. This could be improved by introducing optimization for each training period. 